In [2]:
STOCK_SIZE = 100

In [3]:
def read_instance(path):
    with open(path, 'r') as f:
        orders = [int(x) for x in f.readline().split()]
        return orders

# CPLEX

In [10]:
from docplex.mp.model import Model
from time import perf_counter
import pandas as pd

In [22]:
def cplex_solver(cplex_df, num_instance, orders):
    start = perf_counter()
    
    model = Model(name=f'CSP{num_instance}')
    num_stocks = len(orders)
    num_orders = len(orders)

    order_list = model.integer_var_list(num_orders, lb=orders, ub=orders, name='orders')

    order_placement = model.binary_var_matrix(num_stocks, num_orders, name='order_placement')
    stock_used = model.binary_var_list(num_stocks, name='stock_used')

    for i in range(num_stocks):
        model.add_constraint(sum(order_placement[i,j] * order_list[j] for j in range(num_orders)) <= STOCK_SIZE * stock_used[i])

    for j in range(num_orders):
        model.add_constraint(sum(order_placement[i,j] for i in range(num_stocks)) == 1)

    model.minimize(sum(stock_used))
    
    solution = model.solve()
    
    end = perf_counter() 
    
    print(f'Stocks used for instance {num_instance}: {solution.get_objective_value()}')
    
    cplex_df.loc[0, f'instance {num_instance} num stocks'] = solution.get_objective_value()
    cplex_df.loc[0, f'instance {num_instance} time'] = end-start

In [48]:
orders = read_instance('test_instances/test_instance_11.csp')
initial_value, initial_stocks_used = calc_solution_value(orders)
print(initial_value, initial_stocks_used)
cplex_solver(orders)

775902 317
299.0


In [32]:
cplex_df = pd.DataFrame()

for i in range(15):
    orders = read_instance(f'test_instances/test_instance_{i}.csp')
    cplex_solver(cplex_df, i, orders)

In [70]:
cplex_df

,instance 0 num stocks,instance 0 time,instance 1 num stocks,instance 1 time,instance 2 num stocks,instance 2 time,instance 3 num stocks,instance 3 time,instance 4 num stocks,instance 4 time,...,instance 10 num stocks,instance 10 time,instance 11 num stocks,instance 11 time,instance 12 num stocks,instance 12 time,instance 13 num stocks,instance 13 time,instance 14 num stocks,instance 14 time
0,17,0.425326,20,0.277497,49,16.013496,47,6.825647,33,1.363269,...,76,272.816822,299,54.968373,NaN,NaN,172,904.146431,308,1625.924981


In [35]:
cplex_df.to_csv('cplex.csv')